<a href="https://colab.research.google.com/github/HpDiniz/Leitor-de-PDF/blob/main/PDF_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install tabula-py
!pip install pdfminer
!pip install PyPDF2

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2build2).
0 upgraded, 0 newly installed, 0 to remove and 27 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import io
import re
import os
import cv2
import tabula
import PyPDF2
import pytesseract
import pandas as pd
from PIL import Image
from io import StringIO
from google.colab import files
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter

In [20]:
file_objects = files.upload()

Saving - Posicao Consolidada XP - 55432.pdf to - Posicao Consolidada XP - 55432 (4).pdf


In [21]:
# Converte o conteúdo do PDF para texto
def convert_pdf_to_txt(path):

    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()

    return text

def get_content_of_tables(pdf_file):
    # Extrair todas as tabelas do arquivo PDF
    tables = tabula.read_pdf(pdf_file, multiple_tables=True, pages="all")

    return tables

def get_text_from_page_images(reader):

    # Inicializa variáveis
    texts = []
    page_number = 0

    # Percore todas as páginas do documento
    for page in reader.pages:
        
        page_number += 1

        try:
            # Obtém os recursos da página
            xObject = page['/Resources']['/XObject'].get_object()

            image_number = 0
            # Percorre todos os objetos X da página
            for obj in xObject:

                image_number += 1

                # Verifica se o objeto é uma imagem
                if xObject[obj]['/Subtype'] == '/Image':
                    size = (xObject[obj]['/Width'], xObject[obj]['/Height'])
                    data = xObject[obj].get_data()
                    if xObject[obj]['/ColorSpace'] == '/DeviceRGB':
                        mode = "RGB"
                    else:
                        mode = "P"

                    # Cria uma imagem a partir dos dados
                    img = Image.frombytes(mode, size, data)
                    image_name = file_name[0:25] + "_" + str(page_number) + "_" + str(image_number) + ".png"
                    #image_path = pdf_path = "/content/gdrive/My Drive/Temp/" + image_name

                    # Salva a imagem
                    img.save(image_name)

                    # Carrega a imagem
                    image = cv2.imread(image_name)

                    # Converte a imagem para escala de cinza
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                
                    # Executa o OCR usando o Tesseract
                    text = pytesseract.image_to_string(image).strip()

                    # Adiciona o texto obtido ao array de textos
                    texts.append(text)
        except:
            texts.append(f'Não foi possível ler as imagens da página {page_number}')
    
    return texts

def create_new_df():

    # Cria um DataFrame utilizado no consolidado final
    return pd.DataFrame({
        'CLASSIFICAÇÃO': pd.Series(dtype='str'),
        'ATIVO': pd.Series(dtype='str'),
        'EXPOSIÇÃO': pd.Series(dtype='str'),
        'INSTITUIÇÃO': pd.Series(dtype='str'),
        'ATUAL': pd.Series(dtype='float64'),
        'DATA': pd.Series(dtype='str'),
        'ON/OFF': pd.Series(dtype='str'),
        'DATA DE VENCIMENTO': pd.Series(dtype='str'),
        'INDEXADOR': pd.Series(dtype='str')
    })

def get_tipo_extrato(text_pypdf, text_pdfminer):

    text_1 = text_pypdf[0:100].lower().replace(" ","").replace("\n","")
    text_2 = text_pdfminer[0:100].lower().replace(" ","").replace("\n","")

    if 'extratodecotista' in text_1 and 'extratodecotista' in text_2:
        return "Extrato de Cotista XP"
    elif 'posição&performance' in text_1 and 'posição&performance' in text_2:
        return "Posição e Performance XP"

    return "Posição Consolidada XP"

def obtem_classificacao(ativo):

    # Verifica se é uma ativo no formato de Ticker
    founded = re.search(r'\b[A-Z]{4}(3|4|5|6|11|32|33|34|35)\b', ativo, flags=(re.IGNORECASE))
    if founded:
        return "Ações"
    
    # Define as palavras chave para cada Classificação
    dict_classificacoes = {
        "Ações": [
            "FIC FIA","FIA","FI Ações", "FI Açoes"
        ],
        "Renda Fixa": [
            "Renda Fixa","RF","CDB","LC","LF","LFSN","LFSC","LCA","LCI","CRA","CRI","Deb",
            "Debenture","Debênture","Tesouro","Pré","Pós","Pre","Pos","IPCA","IPCA+","LTN",
            "NTN","NTN-B","NTNB","DI","REF DI","CP","Credito Privado","Crédito Privado",
            "LP","Longo Prazo","Previdencia","Previdência","Prev","VGBL","PGBL"
        ],
        "Multimercado": [
            "FIC FIM","FIM","MM","Multimercado","Multi","COE","Multiestratégia",
            "Multiestratégia"
        ]
    }

    classes = []
    # Percorre cada uma das classificações
    for classificacao in dict_classificacoes:

        # Percorre as palavras-chave de cada uma das classificações
        for key in dict_classificacoes[classificacao]:

            # Verifica se a palavra-chave atual existe no nome do ativo
            founded = re.search(r'\b' + key + r'\b', ativo, flags=(re.IGNORECASE))
            if founded and not classificacao in classes:
                classes.append(classificacao)

    if len(classes) > 0:
        return ' / '.join(classes)

    # Se nenhuma classificação foi encontrada, verifica se é um ETF de investimento estrangeiro
    founded = re.search(r'\b[A-Z]{3}\b', ativo, flags=(re.IGNORECASE))
    if founded:
        return "Ações"

    return "***"

def get_pdf_texts(file):

    text_pypdf = ""
    text_pdfminer = ""

    file_name, file_extension = os.path.splitext(file)

    if file_extension != '.pdf':
        return text_pypdf, text_pdfminer

    # Lê o conteúdo do arquivo PDF
    text_pdfminer = convert_pdf_to_txt(file)
    
    reader = PyPDF2.PdfReader(file)
    page_number = 0

    for page in reader.pages:

        page_number += 1
        text_pypdf = text_pypdf + page.extract_text() #visitor_text=remove_header_and_footer)
    
    # print(get_text_from_page_images(reader))

    return text_pypdf, text_pdfminer

def write_consolidado(df, regex_result, data_emissao, classification = None):
    
    # Preenche o consolidado final
    for res in regex_result:

        # Faça atribuições
        ativo = res[0].replace('\n', ' ').strip()
        data_vencimento = res[1].replace('\n', ' ').strip()
        indexador = res[2].replace('\n', ' ').strip() #.replace('.','')
        valor_bruto = res[3].replace('\n', ' ').strip() #.replace('.','')
        classificacao = classification if classification != None else obtem_classificacao(ativo)
        
        # Se for encontrada alguma data no nome do ativo, algum registro inválido foi coletado
        if re.search(r'\d{2}\/\d{2}\/\d{4}', ativo):
            continue

        # Caso a data de vencimento encontrada, não seja uma data, altere seu valor para "-"
        if not re.search(r'\d{2}\/\d{2}\/\d{4}', data_vencimento):
            data_vencimento = "-"

        # Adiciona os novos valores ao DataFrame
        df = df.append({
            'CLASSIFICAÇÃO': classificacao,
            'ATIVO': ativo,
            'EXPOSIÇÃO': "Real",
            'INSTITUIÇÃO': "XP Investimentos",
            'ATUAL': valor_bruto,
            'DATA': data_emissao,
            'ON/OFF': "ON",
            'DATA DE VENCIMENTO': data_vencimento,
            'INDEXADOR': "-"
        }, ignore_index=True)
    
    return df

In [22]:
def obtem_posicao_performance_xp(text_pypdf):

    # Cria o dataFrame resultante
    df_result = create_new_df()

    # Regex para obter todas as linhas que possuírem ativos de Renda Fixa
    regex = r'\n[\d\/]*([a-zA-Z].+?\s*-\s*[A-Z]{3}\/\d{4})\s*\d{2}\/\d{2}\/\d{4}\s*(\d{2}\/\d{2}\/\d{4}|\-)\s*(\d{2}\/\d{2}\/\d{4})\s*([A-Z\-\+\s]*\s*[\d,.]+%[A-Z\-\s]+)[\d.]+,\d\d\s*[\d.]+,\d\d'
    result = re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))

    for res in result:
        print(res)

    return df_result

def obtem_posicao_consolidada_xp(text_pypdf):

    # Cria o dataFrame resultante
    df_result = create_new_df()

    # Regex para obter a Data de Emissão
    regex = "(?<=Data da Consulta: )\d{2}\/\d{2}\/\d{4}"
    data_emissao = re.search(regex, text_pypdf, flags=(re.IGNORECASE)).group(0)

    # Regex para obter todas as linhas que possuírem ativos de Renda Fixa
    regex = r'\n[\d\/]*([a-zA-Z].+?\s*-\s*[A-Z]{3}\/\d{4})\s*\d{2}\/\d{2}\/\d{4}\s*\d{2}\/\d{2}\/\d{4}\s*(\d{2}\/\d{2}\/\d{4})\s*([A-Z\-\+\s]*\s*[\d,.]+%[A-Z\-\s])*\d+\s+\d+\s*R\$\s*[\d,.]+[.,]\d{2}\s*R\$\s*([\d,.]+[.,]\d{2})\s*R\$\s*[\d,.]+[.,]\d{2}'
    result = re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))

    # Regex para obter todas as linhas que possuírem ativos de Renda Fixa sem carência
    regex = r'\n[\d\/]*([a-zA-Z].+?\s*-\s*[A-Z]{3}\/\d{4})\s*\d{2}\/\d{2}\/\d{4}\s*\-\s*(\d{2}\/\d{2}\/\d{4})\s*([A-Z\-\+\s]*\s*[\d,.]+%[A-Z\-\s])+\d+\s+\d+\s*R\$\s*[\d,.]+[.,]\d{2}\s*R\$\s*([\d,.]+[.,]\d{2})\s*R\$\s*[\d,.]+[.,]\d{2}'
    result = result + re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))

    for res in result:
        print(res)

    # Regex para obter todas as linhas que possuírem ativos de Renda Fixa Pós-Fixados
    regex = r'\n[\d\/]*([a-zA-Z].+?)\s*\d{2}\/\d{2}\/(\d{4})\s*[\d,.]+\s*[\d,.]+\s*R\$\s*[\d,.]+\s*R\$\s*([\d,.]+)\s*R\$\s*[\d,.]+'
    result = result + re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))

    # Regex para obter todas as linhas que possuírem Fundos Imobiliários
    regex = r'([A-Z]{4}(3|4|5|6|11|32|33|34|35))\s+\d+\s+\d+\s+\d+\s+\d+\s+\d+\s+R\$\s*[\d,.]+[.,]\d{2}\s*R\$\s*([\d,.]+[.,]\d{2})'
    result = result + re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))

    # Regex para obter todas as linhas que possuírem COEs
    regex = r'\n[\d\/]*([a-zA-Z].+?)\s*-\s*[\w\s,.]*\s*-\s*\s*\d{2}\.\d{2}\.\d{4}\s*[\w\s,.]*\d{2}\/\d{2}\/\d{4}\s+(\d{2}\/\d{2}\/\d{4})\s+\d+\s*R\$\s*[\d,.]+[.,]\d{2}\s*R\$\s*[\d,.]+[.,]\d{2}\s*R\$\s*([\d,.]+[.,]\d{2})'
    result_coes = re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))

    # Regex para obter todas as linhas que possuírem Ações
    regex = r'([A-Z]{4}(3|4|5|6|11|32|33|34|35))\s+\d+\s+\d+\s+\d+\s+\d+\s+\d+\s+\d+\s+\d+\s+R\$\s*[\d,.]+[.,]\d{2}\s*R\$\s*([\d,.]+[.,]\d{2})'
    result_acoes = re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))

    df_result = write_consolidado(df_result, result, data_emissao)
    df_result = write_consolidado(df_result, result_coes, data_emissao, "Multimercado")
    df_result = write_consolidado(df_result, result_acoes, data_emissao)

    return df_result

def obtem_extrato_cotista_xp(text_pypdf, text_pdfminer):

    # Cria o dataFrame resultante
    df_result = create_new_df()

    # Regex para obter a data de emissão
    regex_0 = "(?<=Movimenta..o de \d{2}\/\d{2}\/\d{4} a )\d{2}\/\d{2}\/\d{4}"
    data_emissao = re.search(regex_0, text_pdfminer, flags=(re.IGNORECASE)).group(0)

    # Regex para obter o texto situado entre o termo "POSIÇÃO CONSOLIDADA" e o termo "Emissão"
    regex_1 = "(?<=POSI..O CONSOLIDADA).*?(?=Emissão:)"
    result_1 = re.search(regex_1, text_pdfminer, flags=(re.IGNORECASE | re.DOTALL)).group(0)

    # Regex para obter todas as linhas que possuírem pelo menos uma letra e um espaço
    regex_2 = "^.*[a-zA-Z] .*$"
    result_2 = re.finditer(regex_2, result_1, flags=(re.MULTILINE))

    # Regex para obter o texto situado entre o termo "POSIÇÃOCONSOLIDADA" e o termo "TotalnaInstituição"
    regex_3 = "(?<=POSI..OCONSOLIDADA\n).*?(?=TotalnaInstituição)"
    result_3 = re.search(regex_3, text_pypdf, flags=(re.IGNORECASE | re.DOTALL)).group(0)

    # Substitui os espaços por um ponto e vírgula, para simular uma tabela CSV
    result_3 = result_3.replace(" ",";")

    # Corrige termos com espaço
    for result in result_2:
        value = result.group(0)
        key = value.replace(" ","")
        result_3 = result_3.replace(key,value)

    # Converte o CSV para o tipo DataFrame
    df = pd.read_csv(StringIO(result_3), sep=";")
    
    # Cria um array de tuplas
    result = []

    # Preenche o consolidado final
    for index, row in df.iterrows():
        result.append((row['Fundo'],"-",row["Valor Bruto"]))

    df_result = write_consolidado(df_result, result, data_emissao)

    return df_result

In [23]:
# Percorre cada um dos arquivos
for file in file_objects.keys():

    df_result = create_new_df()
    text_pypdf, text_pdfminer = get_pdf_texts(file)
    tipo_extrato = get_tipo_extrato(text_pypdf, text_pdfminer)

    if tipo_extrato == "Extrato de Cotista XP":
        df_result = obtem_extrato_cotista_xp(text_pypdf, text_pdfminer)
    elif tipo_extrato == "Posição Consolidada XP":
        df_result = obtem_posicao_consolidada_xp(text_pypdf)
    elif tipo_extrato == "Posição e Performance XP":
        df_result = obtem_posicao_performance_xp(text_pypdf)

    file_name, file_extension = os.path.splitext(file)

    # df_result.to_excel(file_name + ".xlsx", index=False) 
    # files.download(file_name + ".xlsx")

('CDB PINE - NOV/2024', '08/11/2024', '+ 13,05%\n', '112.744,61')
('CDB ORIGINAL - JUN/2025', '21/06/2025', '+ 10,60%\n', '245.774,44')
('NTN-B \n- MAI/2055', '15/05/2055', 'IPC-A + 4,46%\n', '40.569,67')
('NTN-B \n- AGO/2030', '15/08/2030', 'IPC-A + 4,53%\n', '100.212,81')
('NTN-B \n- MAI/2035', '15/05/2035', 'IPC-A + 4,35%\n', '83.870,32')
('NTN-B \n- AGO/2028', '15/08/2028', 'IPC-A + 4,29%\n', '86.440,48')
